<a href="https://colab.research.google.com/github/ajufrancis/S6/blob/main/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

In [35]:
# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define layers including convolutional layers, batch normalization, dropout, and pooling layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # Input: 28x28x1, Output: 28x28x8, RF: 3x3
        self.bn1 = nn.BatchNorm2d(8)
        self.dropout1 = nn.Dropout(0.25)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  # Output: 28x28x16, RF: 5x5
        self.bn2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.25)

        self.pool1 = nn.MaxPool2d(2, 2)  # Output: 14x14x16, RF: 6x6

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)  # Output: 14x14x32, RF: 10x10
        self.bn3 = nn.BatchNorm2d(32)
        self.dropout3 = nn.Dropout(0.25)

        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)  # Output: 14x14x32, RF: 14x14
        self.bn4 = nn.BatchNorm2d(32)
        self.dropout4 = nn.Dropout(0.25)

        self.pool2 = nn.MaxPool2d(2, 2)  # Output: 7x7x32, RF: 16x16

        self.conv5 = nn.Conv2d(32, 32, 3, padding=1)  # Output: 7x7x32, RF: 24x24
        self.bn5 = nn.BatchNorm2d(32)
        self.dropout5 = nn.Dropout(0.25)

        self.gap = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        self.conv6 = nn.Conv2d(32, 10, 1)  # 1x1 Convolution

    def forward(self, x):
        x = self.dropout1(self.bn1(F.relu(self.conv1(x))))
        x = self.dropout2(self.bn2(F.relu(self.conv2(x))))
        x = self.pool1(x)
        x = self.dropout3(self.bn3(F.relu(self.conv3(x))))
        x = self.dropout4(self.bn4(F.relu(self.conv4(x))))
        x = self.pool2(x)
        x = self.dropout5(self.bn5(F.relu(self.conv5(x))))
        x = self.gap(x)
        x = self.conv6(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [36]:
!pip install torchsummary
from torchsummary import summary
# Initialize the model, define hyperparameters, and prepare data loaders
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [37]:


# Data loading and normalization
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=128, shuffle=True, num_workers=4, pin_memory=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [38]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [39]:

# Initialize the network and optimizer
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# Optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)


# Training loop
for epoch in range(1, 21):  # Run for 20 epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()  # Adjust learning rate

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
          Dropout-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 32, 14, 14]           9,248
      BatchNorm2d-12           [-1, 32, 14, 14]              64
          Dropout-13           [-1, 32, 14, 14]               0
        MaxPool2d-14             [-1, 3